# YDB+YandexGPT RAG example

# Installation

First of all, we should install all dependencies and load `.env` file to use `YandexGPT` and `YandexGPTEmbeddings`.

In [1]:
!pip install -qU yandexcloud langchain-community langchain-ydb python-dotenv langchain-text-splitters

In [2]:
import os
from dotenv import load_dotenv
dotenv_path = os.path.join(os.path.dirname(__name__), '.env')
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path)

FOLDER_ID=os.environ.get('YC_FOLDER_ID')

After this step, env variables `YC_FOLDER_ID` and `YC_API_KEY` should be set. Check if you fill `.env` file in case of any errors.

# Prepare data

In this example we will use `Bro Code` by Barney Stinson from `How I Met Your Mother` series.

<div>
<img src="bro_code.jpg" width="200"/>
</div>

Load md file and split in onto langchain's `Document` by header:

In [3]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

with open('bro_code.txt', 'r') as file:
    markdown_document = file.read()

headers_to_split_on = [
    ("#", "Header 1"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on, strip_headers=False)
md_header_splits = markdown_splitter.split_text(markdown_document)
md_header_splits[10]

Document(metadata={'Header 1': 'Статья 11'}, page_content='# Статья 11  \nБратан может попросить его Братана(ов) помочь ему с переездом, но только после\nсогласования и оценки времени обязательства и количества крупных предметов\nмебели. Если Братан значительно недооценил количество своих вещей, то его\nБратаны сохраняют за собой право оставить его имущество как есть – в\nбольшинстве случаев, застрявшем в дверях.')

## Init Embeddings Model and Vector Store

To init `YDB` vector store we should create embeddings model. In this example we will use `YandexGPTEmbeddings` as they work well with russian language.

In [4]:
from langchain_community.embeddings.yandex import YandexGPTEmbeddings
embeddings = YandexGPTEmbeddings(folder_id=FOLDER_ID)

In [5]:
from langchain_ydb.vectorstores import YDB, YDBSettings
config = YDBSettings(
    table="bro_code",
    drop_existing_table=True,
)

vector_store = YDB(embeddings, config=config)

Load prepared documents:

In [6]:
ids = vector_store.add_documents(md_header_splits)
len(ids)

I0000 00:00:1744028132.700810 1735291 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers
Inserting data...: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 131/131 [00:16<00:00,  7.94it/s]


131

Here's a small example how can we use vector store to search for relateable data:

In [7]:
vector_store.similarity_search_with_score("Намечается драка, нужно ли вступиться?")

[(Document(metadata={'Header 1': 'Статья 147'}, page_content='# Статья 147  \nЕсли Братан видит, что другой Братан попал в драку, он должен немедленно стать его\nБратанским прикрытием. Исключения: Если Братан дерется с действительно ужасно\nвыглядящим парнем. Если это третья драка (или больше) его Братана за неделю. Если у\nБратана есть записка от психиатра, освобождающая его от участия в чьем либо\nприкрытии.'),
  0.5891374945640564),
 (Document(metadata={'Header 1': 'Статья 6'}, page_content='# Статья 6  \nБратану не следует валять дурака, если он должен раздеваться до гола перед другими\nБратанами в раздевалке спортзала.\nВЫВОД: Если братан должен пройтись голым в раздевалке спортзала, все другие\nБратаны должны притвориться, что, собственно, ничего необычного в происходящем\nнет, и в то же время, немедленно закрыть (отвести) глаза.'),
  0.5692405700683594),
 (Document(metadata={'Header 1': 'Статья 82'}, page_content='# Статья 82  \nЕсли два Братана вступили в жаркую дискуссию о чем

In [8]:
retriever = vector_store.as_retriever(search_kwargs={"k": 1})

## Make RAG using langchain

We will use `ChatYandexGPT` as a LLM backend because it works well with russian language.

In [9]:
from langchain_community.chat_models import ChatYandexGPT

chat_model = ChatYandexGPT(folder_id=FOLDER_ID)

Finally, let's define our RAG chain:

In [10]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate, ChatPromptTemplate

# Явное создание шаблонов для SystemMessage и HumanMessage
system_template = """Ты - Барни Стинсон. Ты написал неоспоримый кодекс и должен
отвечать на пользовательские вопросы опираясь исключительно на найденные статьи кодекса.
Не придумывай ничего от себя, твой ответ должен базироваться исключительно на предложенной статье кодекса.
Отвечая на вопрос, всегда полностью цитируй статью, а потом подводи короткий итог.

Если найденная статья не связана с вопросом - скажи дословно "Понятия не имею, Братан" без упоминания найденной статьи.
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template = "Статья: {context}\n\nВопрос: {question}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# Объединение шаблонов сообщений в ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    system_message_prompt,
    human_message_prompt
])

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | chat_model
    | StrOutputParser()
)

The defined chain will do:

1. Get question
2. Retrieve data from vector store for additional context 
3. Prepare correct prompt to LLM
4. Invoke prompt on LLM
5. Return output

From this point, we are free to pass any question to our RAG agent:

In [11]:
query = "Братан попросил подбросить его до аэропорта, что делать?"
rag_chain.invoke(query)

'«Братан подбросит другого Братана до аэропорта или обратно, но ни то и другое за одну поездку. Также при этом от него не ожидают прибытия вовремя, помощи с багажом и расспросов о самочувствии и том, как прошёл полёт».\n\nСледует подбросить Братана до аэропорта. Не стоит ожидать своевременного прибытия, помогать с багажом и задавать вопросы о полёте.'

## Examples

In [12]:
query = "Намечается драка, что делать?"
rag_chain.invoke(query)

'«Если Братан видит, что другой Братан попал в драку, он должен немедленно стать его Братанским прикрытием. Исключения: Если Братан дерётся с действительно ужасно выглядящим парнем. Если это третья драка (или больше) его Братана за неделю. Если у Братана есть записка от психиатра, освобождающая его от участия в чьём-либо прикрытии».\n\nТаким образом, если вы стали свидетелем драки или сами в неё втянуты, ваша первоочередная задача — оказать поддержку своему товарищу и стать его прикрытием. Однако есть исключения, которые необходимо учитывать.'

In [13]:
query = "Я купил новую машину, как правильно показать ее друзьям?"
rag_chain.invoke(query)

'Когда Братан, купив новую машину, показывает её своим Братанам, он обязан открыть капот. Следствие — Братаны обязаны присвистнуть в знак одобрения, даже если они не понимают, по поводу чего.\n\nИтог: чтобы правильно показать друзьям новую машину, нужно открыть капот и дождаться одобрительных возгласов от друзей.'

In [14]:
query = "Собираюсь на пьянку, сколько бутылок пива брать с собой?"
rag_chain.invoke(query)

'«Собираясь на вечеринку другого Братана, Братан берёт с собой по крайней мере на одну единицу горючего больше, чем планирует выпить. Таким образом, если по плану идёт 6 единиц пива, он должен принести по крайней мере на одну банку больше. Если на вечеринке слишком много любителей выпить на дармовщину и просто излишек чуваков, Братан вправе оставить свою выпивку при себе, хотя этикет советует ему хотя бы подождать до тех пор, пока этого никто не видит».\n\nИтак, рекомендуется брать с собой на одну бутылку пива больше, чем вы планируете выпить. Однако, если на вечеринке много людей, которые могут попытаться выпить ваше пиво, вы имеете право оставить его при себе.'

In [15]:
query = "Моя девушка спрашивает что было на мальчишнике. Что ей ответить?"
rag_chain.invoke(query)

'«Всё было хорошо». Братан никогда не может брать камеру на мальчишник. (Статья 136)'

In [16]:
query = "Как получить хорошую оценку на ревью?"
rag_chain.invoke(query)

'Понятия не имею, братан.'